Adi Watzman
Nov. 2019
The goal of this notebook is to develop tools to measure trip time end-to-end.
The specific usecase now is to compare trip durations before and after "Nativ +" initiative and analyze the differences (if any).


The main steps:
1. Pick route ids and dates.  
* Query the "Open Bus" splunk database and retrieve siri real time data for these routes in the chosen dates.  
* Query the "Open Bus" splunk database and retrieve GTFS stops locations for these routes.  
* Estimate the time the bus passed in the second** stop and the time it arrived to the last stop, and report their   difference as the trip approximate duration.  

** We use the second stop because it is easier to estimate when a bus passes a stop than estimate the time a bus started its trip. For the first stop there can be many cases such as - bus is very late and drives towards the first stop at the time it was suppose to depart, which can cause false departure time estimation.


In order to retrieve data from our splunk database, I use the [splunk utils](https://github.com/hasadna/open-bus/tree/master/splunk/SplunkUtills) module

# Imports and setup

In [1]:
import getpass
import json
import pandas as pd
import numpy as np
import sys

relative_splunk_utils_path = "../splunk/SplunkUtills"
if relative_splunk_utils_path not in sys.path:
    sys.path.append(relative_splunk_utils_path)

from SplunkUtills import splunk_query_builder, read_splunk

## credentials

In [2]:
with open('/Users/awatzman/Documents/OpenBusDocuments/__credentials.json', 'r') as f:
    credentials = json.load(f)
    
HOST = credentials['host']
PORT = credentials['port']
USERNAME = credentials['username']

In [3]:
# avoid saving the password in a file:
PASSWORD = getpass.getpass()

········


# Templates - Splunk query demo - used for query templates

In [4]:
query = '''search index=route_stats earliest=-20d latest=-10d route_id=5189|
        fields agency_id, route_short_name, route_id, date, all_start_time | '''

GTFS_data = read_splunk(query,
          host =  HOST, port = PORT, username = USERNAME, password = PASSWORD)


start..

your query:
 search index=route_stats earliest=-20d latest=-10d route_id=5189|

        fields agency_id, route_short_name, route_id, date, all_start_time |
 

connection succeed

query status: 100.0%   66 scanned   10 matched   10 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 10



In [11]:
GTFS_data

,agency_id,route_short_name,route_id,date,all_start_time
0,15,501,5189,2019-10-30,05:00:00;05:15:00;05:45:00;06:05:00;06:23:00;0...
1,15,501,5189,2019-10-29,05:00:00;05:15:00;05:45:00;06:05:00;06:23:00;0...
2,15,501,5189,2019-10-28,05:00:00;05:15:00;05:45:00;06:05:00;06:23:00;0...
3,15,501,5189,2019-10-27,05:00:00;05:15:00;05:45:00;06:05:00;06:23:00;0...
4,15,501,5189,2019-10-26,20:00:00;20:01:00;20:15:00;20:30:00;20:45:00;2...
5,15,501,5189,2019-10-25,05:00:00;05:30:00;05:45:00;06:00:00;06:15:00;0...
6,3,501,5189,2019-10-24,05:15:00;05:30:00;05:45:00;05:55:00;06:03:00;0...
7,3,501,5189,2019-10-23,05:15:00;05:30:00;05:45:00;05:55:00;06:03:00;0...


In [12]:
query_kwargs = {
                'index': 'siri',
                'earliest': '-10d', #=last 10 days
                'latest': '-7d',
                #'agency_id': 3,
                'route_short_name': 501,
                'route_id': 5189,
                'planned_start_time': '22:00:00',
               #'max_columns': 100000,
               'columns': ['timestamp','agency_id','route_id','route_short_name','service_id','planned_start_time',
                        'bus_id','predicted_end_time','time_recorded','lat','lon']
                }

SIRI_data = read_splunk(splunk_query_builder(**query_kwargs),
          host =  HOST, port = PORT, username = USERNAME, password = PASSWORD)


start..

your query:
 search index=siri earliest=-10d latest=-7d route_short_name=501 route_id=5189 planned_start_time=22:00:00 |
 fields timestamp, agency_id, route_id, route_short_name, service_id, planned_start_time, bus_id, predicted_end_time, time_recorded, lat, lon |


connection succeed

query status: 100.0%   1769 scanned   87 matched   87 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 87



In [13]:
SIRI_data

,timestamp,agency_id,route_id,route_short_name,service_id,planned_start_time,bus_id,predicted_end_time,time_recorded,lat,lon
0,2019-11-02T22:26:42,15,5189,501,42135488,22:00:00,86343101,22:45:00,22:26:39,32.168880,34.928661
1,2019-11-02T22:25:42,15,5189,501,42135488,22:00:00,86343101,22:44:00,22:25:34,32.168278,34.928368
2,2019-11-02T22:24:42,15,5189,501,42135488,22:00:00,86343101,22:43:00,22:24:20,32.167198,34.927765
3,2019-11-02T22:23:42,15,5189,501,42135488,22:00:00,86343101,22:42:00,22:23:32,32.165466,34.926895
4,2019-11-02T22:22:42,15,5189,501,42135488,22:00:00,86343101,22:45:00,22:22:10,32.164654,34.924950
...,...,...,...,...,...,...,...,...,...,...,...
82,2019-11-02T21:04:42,15,5189,501,42135488,22:00:00,86343101,22:45:00,20:11:31,0.000000,0.000000
83,2019-11-02T21:03:42,15,5189,501,42135488,22:00:00,86343101,22:45:00,20:11:31,0.000000,0.000000
84,2019-11-02T21:02:42,15,5189,501,42135488,22:00:00,86343101,22:45:00,20:11:31,0.000000,0.000000
85,2019-11-02T21:01:42,15,5189,501,42135488,22:00:00,86343101,22:45:00,20:11:31,0.000000,0.000000


# Get Data

## Pick route ids and dates

Use the route_stats and trip_stats [docs](https://open-bus-gtfs-utils.readthedocs.io/en/latest/trip_stats.html) to find relevant data fields.

In [8]:
# human-friendly settings:
route_short_name = '601'
earliest = "10/30/2019:00:00:00"
latest = "10/30/2019:23:59:59"

In [9]:
# find relevant route ids:

get_route_ids_q = '''
search 
    index=route_stats 
    earliest="{earliest}"
    latest="{latest}" 
    route_short_name={route_short_name} |
fields 
    date, 
    agency_id, 
    route_short_name, 
    route_id,
    start_stop_city,
    start_stop_name,
    end_stop_city,
    end_stop_name,
    all_start_time | 
'''.format(earliest=earliest, latest=latest, route_short_name=route_short_name)

route_ids = read_splunk(get_route_ids_q,
                        host=HOST, port=PORT, username=USERNAME, password=PASSWORD)


start..

your query:
 
search 
    index=route_stats 
    earliest="10/30/2019:00:00:00"
    latest="10/30/2019:23:59:59" 
    route_short_name=601 |

fields 
    date, 
    agency_id, 
    route_short_name, 
    route_id,
    start_stop_city,
    start_stop_name,
    end_stop_city,
    end_stop_name,
    all_start_time |
 


connection succeed

query status: 100.0%   49 scanned   3 matched   3 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 3



In [10]:
route_ids

,date,agency_id,route_short_name,route_id,start_stop_city,start_stop_name,end_stop_city,end_stop_name,all_start_time
0,2019-10-30,15,601,18077,תל אביב יפו,ת.מרכזית ת''א ק.6/רציפים,נתניה,ת. רכבת נתניה,19:35:00;20:05:00;20:40:00;21:20:00;21:35:00;2...
1,2019-10-30,15,601,700,תל אביב יפו,ת.מרכזית ת''א ק.6/רציפים,נתניה,ת. רכבת נתניה,05:40:00;06:00:00;06:15:00;06:30:00;06:40:00;0...
2,2019-10-30,15,601,698,נתניה,ת. רכבת נתניה,תל אביב יפו,ת. מרכזית ת''א ק. 6/הורדה,05:30:00;06:00:00;06:20:00;06:40:00;06:50:00;0...


In [17]:
#route_ids.all_start_time.str.split(';').apply(lambda x: sorted(x)).values

array([list(['19:35:00', '20:05:00', '20:40:00', '21:20:00', '21:35:00', '21:50:00', '22:10:00', '22:30:00', '22:50:00', '23:10:00', '23:35:00']),
       list(['05:40:00', '06:00:00', '06:15:00', '06:30:00', '06:40:00', '06:50:00', '07:00:00', '07:10:00', '07:20:00', '07:40:00', '07:50:00', '08:00:00', '08:12:00', '08:24:00', '08:36:00', '08:50:00', '09:00:00', '09:10:00', '09:20:00', '09:30:00', '09:45:00', '10:00:00', '10:20:00', '10:40:00', '11:00:00', '11:20:00', '11:40:00', '12:00:00', '12:35:00', '13:00:00', '13:30:00', '14:00:00', '14:30:00', '14:45:00', '15:00:00', '15:20:00', '15:40:00', '16:00:00', '16:20:00', '16:40:00', '17:00:00', '17:25:00', '17:45:00', '18:00:00', '18:25:00', '18:45:00', '19:00:00']),
       list(['05:30:00', '06:00:00', '06:20:00', '06:40:00', '06:50:00', '07:00:00', '07:10:00', '07:20:00', '07:30:00', '07:40:00', '07:50:00', '08:00:00', '08:20:00', '08:40:00', '09:00:00', '09:15:00', '09:30:00', '09:45:00', '10:05:00', '10:25:00', '10:45:00', '11:05:00

Choose the relevat route id and continue:

In [11]:
route_id = '698' # Netanya -> TLV

## Retreive SIRI data

In [12]:
siri_cols = ['timestamp','agency_id','route_id','route_short_name','service_id','planned_start_time',
             'bus_id','predicted_end_time','time_recorded','lat','lon']
query_kwargs = {
                'index': 'siri',
                'earliest': earliest, 
                'latest': latest,
                'route_id': route_id,
                'columns': siri_cols
                }

siri_data = read_splunk(splunk_query_builder(**query_kwargs),
                        host=HOST, port=PORT, username=USERNAME, password=PASSWORD)

start..

your query:
 search index=siri earliest=10/30/2019:00:00:00 latest=10/30/2019:23:59:59 route_id=698 |
 fields timestamp, agency_id, route_id, route_short_name, service_id, planned_start_time, bus_id, predicted_end_time, time_recorded, lat, lon |


connection succeed

query status: 100.0%   7065 scanned   6391 matched   6391 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 6391



In [13]:
print(siri_data.shape)
siri_data.head()

(6391, 11)


,timestamp,agency_id,route_id,route_short_name,service_id,planned_start_time,bus_id,predicted_end_time,time_recorded,lat,lon
0,2019-10-30T23:41:03,15,698,601,42325925,22:35:00,47091901,23:42:00,23:41:06,32.061081,34.784721
1,2019-10-30T23:40:03,15,698,601,42325925,22:35:00,47091901,23:41:00,23:40:05,32.063702,34.785473
2,2019-10-30T23:39:03,15,698,601,42325925,22:35:00,47091901,23:41:00,23:39:08,32.065762,34.786610
3,2019-10-30T23:38:03,15,698,601,42325925,22:35:00,47091901,23:41:00,23:38:08,32.071716,34.790062
4,2019-10-30T23:37:03,15,698,601,42325925,22:35:00,47091901,23:40:00,23:36:48,32.074947,34.790810


## Retreive stops locations (GTFS)

Continue from here - get the all_stop_latlon from route_stats of the relevant route!.   
https://open-bus-gtfs-utils.readthedocs.io/en/latest/index.html

In [27]:
stops_cols = ['route_id', 'date', 'all_stop_id', 'all_stop_code',  'all_stop_latlon']
stops_query_kwargs = {
                      'index': 'route_stats',
                      'earliest': earliest, 
#                       'latest': latest,
                      'latest': "10/31/2019:23:59:59",
                      'route_id': route_id,
                      'columns': stops_cols
                      }

stops_data = read_splunk(splunk_query_builder(**stops_query_kwargs),
                         host=HOST, port=PORT, username=USERNAME, password=PASSWORD)

start..

your query:
 search index=route_stats earliest=10/30/2019:00:00:00 latest=10/31/2019:23:59:59 route_id=698 |
 fields route_id, date, all_stop_id, all_stop_code, all_stop_latlon |


connection succeed

query status: 100.0%   24 scanned   2 matched   2 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 2



In [31]:
stops_data # one row per route id and date

,route_id,date,all_stop_id,all_stop_code,all_stop_latlon
0,698,2019-10-31,24323;333;358;42546;324;320;309;307;302;291;28...,36875;39247;39273;32196;39237;39233;39222;3922...,"32.319671,34.868105;32.325694,34.864841;32.326..."
1,698,2019-10-30,24323;333;358;42546;324;320;309;307;302;291;28...,36875;39247;39273;32196;39237;39233;39222;3922...,"32.319671,34.868105;32.325694,34.864841;32.326..."


In [57]:
def get_stops_df_per_route_date(row, stops_seq_cols=['all_stop_id', 'all_stop_code', 'all_stop_latlon']):
    """ Create dataframe with a row for each stop
    Input: row - a row of a df generated by the above splunk stops query.
    """
    stops_dict = row.to_dict()
    
    # split each cell with stops sequence:
    for col in stops_seq_cols:
        stops_dict[col] = stops_dict[col].split(';')
    
    # enumerate the stops in the sequence:
    seq_len = len(stops_dict[stops_seq_cols[0]])
    stops_dict['stop_order'] = list(range(seq_len))
    
    return pd.DataFrame(stops_dict)

In [58]:
# reshape the stops_data df to have one row per route_id, date and stop:
stops_data2 = pd.concat([get_stops_df_per_route_date(row) for ind, row in stops_data.iterrows()], axis=0)
stop_data

,route_id,date,all_stop_id,all_stop_code,all_stop_latlon,stop_order
0,698,2019-10-31,24323,36875,"32.319671,34.868105",0
1,698,2019-10-31,333,39247,"32.325694,34.864841",1
2,698,2019-10-31,358,39273,"32.326819,34.862473",2
3,698,2019-10-31,42546,32196,"32.326629,34.857538",3
4,698,2019-10-31,324,39237,"32.324201,34.856548",4
...,...,...,...,...,...,...
42,698,2019-10-30,13103,21472,"32.074914,34.790734",42
43,698,2019-10-30,13206,21675,"32.07189,34.789291",43
44,698,2019-10-30,39960,20143,"32.066005,34.786424",44
45,698,2019-10-30,13277,22947,"32.061357,34.78463",45


## TODO - Pick the 2nd and last stops for each route id and date, estimate the time in these stops using SIRI data, and claculate the time difference.

## TODO - run on larger scale - to get statistics.

## TODO - convert the important code parts to a generic module